<a href="https://colab.research.google.com/github/copyrightFreeGenAI/copyrightFreeImagesGenAI/blob/main/3.%20Fine-Tuning/Generate_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Set Paths
annValInstanceFiles = "/content/gdrive/MyDrive/annotations2014/instances_train2014.json" # Path to benchamrk instance json
annValCaptionFiles = "/content/gdrive/MyDrive/annotations2014/captions_train2014.json" # Path to benchmark caption json
source_directory = "/content/gdrive/MyDrive/DATA/COCO/Validation/Images" # Path to directory where benchmark MS COCO images are saved

DREAM_WORKSPACE = '/content/gdrive/MyDrive/Fast-Dreambooth/Sessions' # Path to DreamBooth sessions
DREAM_target_directory = "/content/gdrive/MyDrive/Dream/Dream" # Path to store DreamBooth Images
LORA_WORKSPACE = '/content/gdrive/MyDrive/LoRA' # Path to LoRA sessions
LORA_target_directory = "/content/gdrive/MyDrive/LoRA/LoRA" # Path to store LoRA Images
TI_WORKSPACE = '/content/gdrive/MyDrive/Textual-Inversion' # Path to Textual Inversion sessions
TI_target_directory = "/content/gdrive/MyDrive/Textual-Inversion/Textual-Inversion" # Path to store TI Images

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from pycocotools.coco import COCO
import os
from diffusers import StableDiffusionPipeline, DiffusionPipeline

In [ ]:
coco=COCO(annValInstanceFiles)
coco_caps=COCO(annValCaptionFiles)
cats = coco.loadCats(coco.getCatIds())

# Generate DreamBooth

In [ ]:
for cat in cats:
  print(cat['name'])
  category = cat['name'].replace(" ", "_")
  img_taget = os.path.join(DREAM_target_directory, cat['name'])
  checkpoint_path = os.path.join(DREAM_WORKSPACE, category, category+".ckpt")
  if not os.path.exists(checkpoint_path):
    continue
  catIds = coco.getCatIds(catNms=[cat['name']])
  imgIds = coco.getImgIds(catIds=catIds)
  filenames = os.listdir(os.path.join(source_directory, cat['name']))
  names_before_dot = [int(os.path.splitext(file)[0]) for file in filenames]
  os.makedirs(img_taget, exist_ok=True)
  if len(os.listdir(img_taget)) != 100:
    pipeline = StableDiffusionPipeline.from_single_file(checkpoint_path, config="Mitsua/mitsua-diffusion-one", safety_checker=None).to("cuda")
    for image_id in names_before_dot:
      img_path = os.path.join(img_taget, str(image_id) + ".jpg")
      if os.path.exists(img_path):
            continue
      img = coco.loadImgs(imgIds[image_id])[0]
      annIds = coco_caps.getAnnIds(imgIds=img['id'])
      anns = coco_caps.loadAnns(annIds)
      captions_list = [item['caption'].lower() for item in anns]
      prompt = "qwer. " + ' '.join(captions_list)
      prompt = prompt.replace(cat['name'], "qwer")
      image = pipeline(prompt).images[0]
      image.save(img_path)
    del pipeline

# Generate LoRA

In [ ]:
for cat in cats:
  print(cat['name'])
  category = cat['name'].replace(" ", "_")
  img_taget = os.path.join(LORA_target_directory, cat['name'])
  checkpoint_path = os.path.join(LORA_WORKSPACE, category, 'output', category+"-20.safetensors")
  if not os.path.exists(checkpoint_path):
    continue
  catIds = coco.getCatIds(catNms=[cat['name']])
  imgIds = coco.getImgIds(catIds=catIds)
  filenames = os.listdir(os.path.join(source_directory, cat['name']))
  names_before_dot = [int(os.path.splitext(file)[0]) for file in filenames]
  os.makedirs(img_taget, exist_ok=True)
  if len(os.listdir(img_taget)) != 100:
    pipeline = DiffusionPipeline.from_pretrained("Mitsua/mitsua-diffusion-one", safety_checker=None).to("cuda")
    pipeline.load_lora_weights(checkpoint_path)
    for image_id in names_before_dot:
      img_path = os.path.join(img_taget, str(image_id) + ".jpg")
      if os.path.exists(img_path):
            continue
      img = coco.loadImgs(imgIds[image_id])[0]
      annIds = coco_caps.getAnnIds(imgIds=img['id'])
      anns = coco_caps.loadAnns(annIds)
      captions_list = [item['caption'].lower() for item in anns]
      prompt = "qwer. " + ' '.join(captions_list)
      prompt = prompt.replace(cat['name'], "qwer")
      image = pipeline(prompt).images[0]
      image.save(img_path)
    del pipeline

# Generate Textual Inversion

In [ ]:
for category in os.listdir(source_directory):
  print(category)
  img_target = os.path.join(TI_target_directory, category)
  os.makedirs(img_target, exist_ok=True)
  if len(os.listdir(img_target)) == 100:
    continue
  dir_path = os.path.join(TI_WORKSPACE, category, 'learned_embeds.bin')
  if not os.path.exists(dir_path):
    continue
  pipeline = DiffusionPipeline.from_pretrained("Mitsua/mitsua-diffusion-one", safety_checker = None).to("cuda")
  pipeline.load_textual_inversion(dir_path)
  catIds = coco.getCatIds(catNms=[category])
  imgIds = coco.getImgIds(catIds=catIds)
  filenames = os.listdir(os.path.join(source_directory, category))
  names_before_dot = [int(os.path.splitext(file)[0]) for file in filenames]
  for image_id in names_before_dot:
      img_path = os.path.join(img_target, str(image_id) + ".jpg")
      if os.path.exists(img_path):
        continue
      img = coco.loadImgs(imgIds[image_id])[0]
      annIds = coco_caps.getAnnIds(imgIds=img['id'])
      anns = coco_caps.loadAnns(annIds)
      captions_list = [item['caption'].lower() for item in anns]
      prompt = category + ' '.join(captions_list)
      prompt = prompt.replace(category, f"<{category}>")
      image = pipeline(prompt).images[0]
      image.save(img_path)
  del pipeline